In [28]:
import pandas as pd
import requests
import numpy as np

def retrieve_event_data():
    df = pd.read_csv('http://data.gdeltproject.org/gdeltv2/20231113233000.export.CSV.zip',sep='\t')
    df = df.iloc[:,[0,1,26,27,28,52,53,56,57,59,60]]
    df.columns = ['GLOBALEVENTID', 'SQLDATE', 'EventCode', 'EventBaseCode','EventRootCode','ActionGeo_FullName','ActionGeo_CountryCode','ActionGeo_Lat','ActionGeo_Long','DATEADDED','SOURCEURL']
    return df

,GLOBALEVENTID,SQLDATE,EventCode,EventBaseCode,EventRootCode,ActionGeo_FullName,ActionGeo_CountryCode,ActionGeo_Lat,ActionGeo_Long,DATEADDED,SOURCEURL
0,1140018114,20221113,173,173,17,Argentina,AR,-34.000000,-64.000000,20231113233000,https://www.plenglish.com/news/2023/11/13/arge...
1,1140018115,20221113,1821,182,18,"Brown University, Rhode Island, United States",US,41.826200,-71.397600,20231113233000,https://turnto10.com/i-team/sexual-assault-rap...
2,1140018116,20221113,1821,182,18,"Brown University, Rhode Island, United States",US,41.826200,-71.397600,20231113233000,https://turnto10.com/i-team/sexual-assault-rap...
3,1140018117,20221113,1821,182,18,"Brown University, Rhode Island, United States",US,41.826200,-71.397600,20231113233000,https://turnto10.com/i-team/sexual-assault-rap...
4,1140018118,20221113,84,84,8,NaN,NaN,NaN,NaN,20231113233000,https://www.extremetech.com/mobile/samsung-may...
5,1140018119,20221113,83,83,8,"Jeli, Kelantan, Malaysia",MY,5.587620,101.810000,20231113233000,https://www.thestar.com.my/news/nation/2023/11...
6,1140018120,20221113,20,20,2,"Moscow, Moskva, Russia",RS,55.752200,37.615600,20231113233000,https://www.batonrougepost.com/news/274031872/...
7,1140018121,20221113,20,20,2,"Moscow, Moskva, Russia",RS,55.752200,37.615600,20231113233000,https://www.batonrougepost.com/news/274031872/...
8,1140018122,20221113,871,87,8,"Balochistan, Balochistan, Pakistan",PK,28.478500,65.643600,20231113233000,https://www.bignewsnetwork.com/news/274031898/...
9,1140018123,20221113,1821,182,18,"Brown University, Rhode Island, United States",US,41.826200,-71.397600,20231113233000,https://turnto10.com/i-team/sexual-assault-rap...


In [ ]:
def flatten_coordinates(coord_list):
    """Recursive function to unnest all coordinates.
    
    Parameters:
    coord_list: A list of coordinates
    """
    flattened = []
    for item in coord_list:
        if isinstance(item, list):
            flattened.extend(flatten_coordinates(item))
        else:
            flattened.append(item)
    return flattened
    
def retrieve_us_coordinates():
    url = 'https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json'
    response = requests.get(url)

    geojson_data = response.json()
    all_coordinates = []
    for x in geojson_data['features']:
        geometry = x.get('geometry',{})
        coordinates = geometry.get('coordinates',{})

        if coordinates and geometry['type'] == 'Polygon':
            if isinstance(coordinates, list):
                all_coordinates.extend(coordinates)
            else:
                all_coordinates.append(coordinates)

    flat_coordinates = flatten_coordinates(all_coordinates)
    pairs = [(flat_coordinates[i+1], flat_coordinates[i]) for i in range(0, len(flat_coordinates), 2)]
    return pd.DataFrame(pairs, columns=['Latitude','Longitude'])

In [ ]:
def filter_df(event_data_df, us_coord_df, merge=False):
    """
    I would merge here but using the US coordinates retrieved from the json file returned an empty df.
    Instead, I'll keep the option open of using those coordinates and filter on the 'ActionGeo_CountryCode' column.
    
    Parameters:
    event_data_df: DataFrame of event data
    us_coord_df: DataFrame of US coodinates
    merge: Boolean to determine if us_coord_df will be used to filter event_data_df
    """
    if merge:
        return pd.merge(event_data_df, us_coord_df, left_on=['ActionGeo_Lat','ActionGeo_Long'], right_on=['Latitude','Longitude'])
    else:
        return event_data_df[event_data_df['ActionGeo_CountryCode'] == 'US']

In [ ]:
def prep_data(df, event_root_code_dic, event_base_code_dic,event_code_dic,map_to_iso2_dic):
    """
    Replace values in DataFrame columns with appropriate mappings.
    
    Parameters:
    df: DataFrame to be prepped
    event_root_code_dic: Dictionary of event root code values
    event_base_code_dic: Dictionary of event base code values
    event_code_dic: Dictionary of event code values
    map_to_iso2_dic: Dictionary of iso2 values
    """
    df['EventRootCode'] = df['EventRootCode'].replace(event_root_code_dic)
    df['EventBaseCode'] = df['EventBaseCode'].replace(event_base_code_dic)
    df['EventCode'] = df['EventCode'].replace(event_code_dic)
    df['ActionGeo_CountryCode'] = df['ActionGeo_CountryCode'].replace(map_to_iso2_dic)
    return df

In [30]:
event_root_codes = {
'14': 'PROTESTS',
'18': 'ASSAULT',
'19': 'FIGHT',
'20': 'USE UNCONVENTIONAL MASS VIOLENCE'
}
event_base_codes = {
'140': 'Engage in political dissent, not specified below',
'141': 'Demonstrate or rally, not specified below',
'142': 'Conduct hunger strike, not specified below',
'143': 'Conduct strike or boycott, not specified below',
'144': 'Obstruct passage, block, not specified below',
'145': 'Protest violently, riot, not specified below',
'180': 'Use unconventional violence, not specified below',
'181': 'Abduct, hijack, or take hostage',
'182': 'Physically assault, not specified below',
'183': 'Conduct suicide, car, or other non-military bombing, not specified below',

'184': 'Use as human shield',
'185': 'Attempt to assassinate',
'186': 'Assassinate',
'190': 'Use conventional military force, not specified below',
'191': 'Impose blockade, restrict movement',
'192': 'Occupy territory',
'193': 'Fight with small arms and light weapons',
'194': 'Fight with artillery and tanks',
'195': 'Employ aerial weapons, not specified below',
'196': 'Violate ceasefire',
'200': 'Use unconventional mass violence, not specified below',
'201': 'Engage in mass expulsion',
'202': 'Engage in mass killings',
'203': 'Engage in ethnic cleansing',
'204': 'Use weapons of mass destruction, not specified below'
}
event_codes = {
'140': 'Engage in political dissent, not specified below',
'141': 'Demonstrate or rally, not specified below',
'1411': 'Demonstrate for leadership change',
'1412': 'Demonstrate for policy change',
'1413': 'Demonstrate for rights',
'1414': 'Demonstrate for change in institutions, regime',
'142': 'Conduct hunger strike, not specified below',
'1421': 'Conduct hunger strike for leadership change',
'1422': 'Conduct hunger strike for policy change',
'1423': 'Conduct hunger strike for rights',
'1424': 'Conduct hunger strike for change in institutions, regime',
'143': 'Conduct strike or boycott, not specified below',
'1431': 'Conduct strike or boycott for leadership change',
'1432': 'Conduct strike or boycott for policy change',
'1433': 'Conduct strike or boycott for rights',
'1434': 'Conduct strike or boycott for change in institutions, regime',
'144': 'Obstruct passage, block, not specified below',
'1441': 'Obstruct passage to demand leadership change',
'1442': 'Obstruct passage to demand policy change',
'1443': 'Obstruct passage to demand rights',
'1444': 'Obstruct passage to demand change in institutions, regime',
'145': 'Protest violently, riot, not specified below',
'1451': 'Engage in violent protest for leadership change',

'1452': 'Engage in violent protest for policy change',
'1453': 'Engage in violent protest for rights',
'1454': 'Engage in violent protest for change in institutions, regime',
'180': 'Use unconventional violence, not specified below',
'181': 'Abduct, hijack, or take hostage',
'182': 'Physically assault, not specified below',
'1821': 'Sexually assault',
'1822': 'Torture',
'1823': 'Kill by physical assault',
'183': 'Conduct suicide, car, or other non-military bombing, not specified below',
'1831': 'Carry out suicide bombing',
'1832': 'Carry out vehicular bombing',
'1833': 'Carry out roadside bombing',
'1834': 'Carry out location bombing',
'184': 'Use as human shield',
'185': 'Attempt to assassinate',
'186': 'Assassinate',
'190': 'Use conventional military force, not specified below',
'191': 'Impose blockade, restrict movement',
'192': 'Occupy territory',
'193': 'Fight with small arms and light weapons',
'194': 'Fight with artillery and tanks',
'195': 'Employ aerial weapons, not specified below',
'1951': 'Employ precision-guided aerial munitions',
'1952': 'Employ remotely piloted aerial munitions',
'196': 'Violate ceasefire',
'200': 'Use unconventional mass violence, not specified below',
'201': 'Engage in mass expulsion',
'202': 'Engage in mass killings',
'203': 'Engage in ethnic cleansing',
'204': 'Use weapons of mass destruction, not specified below',
'2041': 'Use chemical, biological, or radiological weapons',
'2042': 'Detonate nuclear weapons'
}
map_fips_to_iso2 = {
'AF': 'AF', 'AX': np.NaN, 'AL': 'AL', 'AG': 'DZ', 'AQ': 'AS', 'AN': 'AD', 'AO': 'AO', 'AV': 'AI', 'AY': 'AQ',
'AC': 'AG',
'AR': 'AR', 'AM': 'AM', 'AA': 'AW', 'AT': 'AU', 'AS': 'AU', 'AU': 'AT', 'AJ': 'AZ', 'BF': 'BS', 'BA': 'BH', 'FQ':
'UM',

'BG': 'BD', 'BB': 'BB', 'BS': 'RE', 'BO': 'BY', 'BE': 'BE', 'BH': 'BZ', 'BN': 'BJ', 'BD': 'BM', 'BT': 'BT', 'BL':
'BO',
'BK': 'BA', 'BC': 'BW', 'BV': 'BV', 'BR': 'BR', 'IO': 'IO', 'BX': 'BN', 'BU': 'BG', 'UV': 'BF', 'BM': 'MM', 'BY':
'BI',
'CB': 'KH', 'CM': 'CM', 'CA': 'CA', 'CV': 'CV', 'CJ': 'KY', 'CT': 'CF', 'CD': 'TD', 'CI': 'CL', 'CH': 'CN', 'KT':
'CX',
'IP': 'PF', 'CK': 'CC', 'CO': 'CO', 'CN': 'KM', 'CG': 'CD', 'CF': 'CG', 'CW': 'CK', 'CR': 'AU', 'CS': 'CR',
'IV': 'CI',
'HR': 'HR', 'CU': 'CU', 'UC': 'CW', 'CY': 'CY', 'EZ': 'CZ', 'DA': 'DK', 'DX': np.NaN, 'DJ': 'DJ', 'DO': 'DM',
'DR': 'DO',
'EC': 'EC', 'EG': 'EG', 'ES': 'SV', 'EK': 'GQ', 'ER': 'ER', 'EN': 'EE', 'ET': 'ET', 'PJ': np.NaN, 'EU': 'RE',
'FK': 'FK',
'FO': 'FO', 'FJ': 'FJ', 'FI': 'FI', 'FR': 'FR', 'FG': 'GF', 'FP': 'PF', 'FS': 'TF', 'GB': 'GA', 'GA': 'GM', 'GZ':
'PS',
'GG': 'GE', 'GM': 'DE', 'GH': 'GH', 'GI': 'GI', 'GO': 'RE', 'GR': 'GR', 'GL': 'GL', 'GJ': 'GD', 'GP': 'GP',
'GQ': 'GU',
'GT': 'GT', 'GK': 'GB', 'GV': 'GN', 'PU': 'GW', 'GY': 'GY', 'HA': 'HT', 'HM': 'HM', 'HO': 'HN', 'HK': 'HK',
'HQ': 'UM',
'HU': 'HU', 'IC': 'IS', 'IN': 'IN', 'ID': 'ID', 'IR': 'IR', 'IZ': 'IQ', 'EI': 'IE', 'IM': 'GB', 'IS': 'IL', 'IT': 'IT',
'JM': 'JM', 'JN': 'SJ', 'JA': 'JP', 'DQ': 'UM', 'JE': 'GB', 'JQ': 'UM', 'JO': 'JO', 'JU': 'RE', 'KZ': 'KZ', 'KE':
'KE',
'KQ': 'UM', 'KR': 'KI', 'KN': 'KP', 'KS': 'KR', 'KV': np.NaN, 'KU': 'KW', 'KG': 'KG', 'LA': 'LA', 'LG': 'LV',
'LE': 'LB',
'LT': 'LS', 'LI': 'LR', 'LY': 'LY', 'LS': 'LI', 'LH': 'LT', 'LU': 'LU', 'MC': 'MO', 'MK': 'MK', 'MA': 'MG', 'MI':
'MW',
'MY': 'MY', 'MV': 'MV', 'ML': 'ML', 'MT': 'MT', 'RM': 'MH', 'MB': 'MQ', 'MR': 'MR', 'MP': 'MU', 'MF': 'YT',
'MX': 'MX',
'FM': 'FM', 'MQ': 'UM', 'MD': 'MD', 'MN': 'MC', 'MG': 'MN', 'MJ': 'ME', 'MH': 'MS', 'MO': 'MA', 'MZ':
'MZ', 'WA': 'NA',
'NR': 'NR', 'BQ': 'UM', 'NP': 'NP', 'NL': 'NL', 'NC': 'NC', 'NZ': 'NZ', 'NU': 'NI', 'NG': 'NE', 'NI': 'NG', 'NE':
'NU',
'NF': 'NF', 'CQ': 'MP', 'NO': 'NO', 'MU': 'OM', 'PK': 'PK', 'PS': 'PW', 'LQ': 'UM', 'PM': 'PA', 'PP': 'PG',
'PF': np.NaN,
'PA': 'PY', 'PE': 'PE', 'RP': 'PH', 'PC': 'PN', 'PL': 'PL', 'PO': 'PT', 'RQ': 'PR', 'QA': 'QA', 'RE': 'RE', 'RO':
'RO',
'RS': 'RU', 'RW': 'RW', 'TB': 'BL', 'SH': 'SH', 'SC': 'KN', 'ST': 'LC', 'RN': 'MF', 'SB': 'PM', 'VC': 'VC',
'WS': 'WS',
'SM': 'SM', 'TP': 'ST', 'SA': 'SA', 'SG': 'SN', 'RI': 'RS', 'SE': 'SC', 'SL': 'SL', 'SN': 'SG', 'NN': 'SX', 'LO':
'SK',
'SI': 'SI', 'BP': 'SB', 'SO': 'SO', 'SF': 'ZA', 'SX': 'GS', 'OD': 'SS', 'SP': 'ES', 'PG': np.NaN, 'CE': 'LK',
'SU': 'SD',
'NS': 'SR', 'SV': 'SJ', 'WZ': 'SZ', 'SW': 'SE', 'SZ': 'CH', 'SY': 'SY', 'TW': 'TW', 'TI': 'TJ', 'TZ': 'TZ', 'TH':
'TH',
'TT': 'TL', 'TO': 'TG', 'TL': 'TK', 'TN': 'TO', 'TD': 'TT', 'TE': 'UM', 'TS': 'TN', 'TU': 'TR', 'TX': 'TM', 'TK':
'TC',

'TV': 'TV', 'UG': 'UG', 'UP': 'UA', 'AE': 'AE', 'UK': 'GB', 'US': 'US', 'UY': 'UY', 'UZ': 'UZ', 'NH': 'VU', 'VT':
'VA',
'VE': 'VE', 'VM': 'VN', 'VI': 'VG', 'VQ': 'VI', 'WQ': 'UM', 'WF': 'WF', 'WE': 'PS', 'WI': 'EH', 'YM': 'YE',
'ZA': 'ZM',
'ZI': 'ZW'}

In [ ]:
def data_quality_check(df):
    """
    Check DataFrame for data quality.
    
    Parameters:
    df: DataFrame to be checked
    """
    # Check we're getting right number of columns
    assert len(df.columns) == 11,f"Expecting 11 columns. Received {len(df.columns)}"
    # Check if any rows are null
    assert df.isnull().sum() == 0,"At least one row is null. Check data."
    # Check all values in primary key column are unique
    assert df['GLOBALEVENTID'].nunique() == len(df['GLOBALEVENTID']),"There are duplicate values in GLOBALEVENTID column"

In [ ]:
def load_data(df, table_name, database_url):
    """
    Load data into a Postgres table.
    
    Parameters: 
    df: DataFrame to be loaded
    table_name: Name of the PostgresSQL table
    database_url: Connection URL for the PostgresSQL database
    """
    engine = create_enginer(database_url)
    
    try:
        df.to_sql(table_name, con=engine, if_exists='replace', index=False)
        print(f"Data successfully loaded into '{table_name}' table.")
    except Exception as e:
        print(f"Error loading data into '{table_name}' table: {e}")
    finally:
        # Close the database connection
        engine.dispose()